In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import requests
import math
import time

SUNRISE = 15
DAY_LENGTH = 30
TICKS_PER_DAY = 60

# Add data loading function
def load_historical_data(days=1000):
    """Load and aggregate historical price data"""
    BASE_URL = "https://icelec50015.azurewebsites.net"
    
    def fetch_day_data():
        response = requests.get(f"{BASE_URL}/yesterday")
        return pd.DataFrame(response.json())
    
    buy_prices = []
    sell_prices = []
    demands = []

    for _ in range(days):
        df = fetch_day_data()
        buy_prices.append(df['buy_price'].values)
        sell_prices.append(df['sell_price'].values)
        demands.append(df['demand'].values)
    
    buy_prices = np.array(buy_prices)
    sell_prices = np.array(sell_prices)
    demands = np.array(demands)
    
    # Calculate averages
    df_nn = pd.DataFrame({
        'avg_buy_price': buy_prices.mean(axis=0),
        'avg_sell_price': sell_prices.mean(axis=0),
        'demand': demands.mean(axis=0)
    })
    
    return df_nn

# Enhanced DualPredictionNet with improved architecture
class EnhancedDualPredictionNet(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        
        # Shared encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.2)
        )
        
        # Price prediction branch
        self.price_head = nn.Sequential(
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 2)  # Predict both buy and sell prices
        )
        
        # Demand prediction branch
        self.demand_head = nn.Sequential(
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )
        
    def forward(self, x):
        shared_features = self.encoder(x)
        prices = self.price_head(shared_features)  # [buy_price, sell_price]
        demand = self.demand_head(shared_features)
        return prices, demand

# Prepare training data
def prepare_training_data(df):
    # Feature engineering
    df['time_of_day'] = np.arange(len(df)) % 60
    df['time_sin'] = np.sin(2 * np.pi * df['time_of_day'] / 60)
    df['time_cos'] = np.cos(2 * np.pi * df['time_of_day'] / 60)
    
    # Add sunlight calculation
    df['sunlight'] = df['time_of_day'].apply(lambda x: 
        int(math.sin((x-SUNRISE)*math.pi/DAY_LENGTH)*100) 
        if SUNRISE <= x < SUNRISE + DAY_LENGTH else 0
    )
    
    # Add rolling statistics
    for window in [3, 5, 10]:
        df[f'price_ma_{window}'] = df['avg_buy_price'].rolling(window).mean()
        df[f'demand_ma_{window}'] = df['demand'].rolling(window).mean()
        df[f'price_std_{window}'] = df['avg_buy_price'].rolling(window).std()
    
    # Fill NaN values
    df = df.fillna(method='bfill')
    
    return df

# Training function
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=100):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    
    best_val_loss = float('inf')
    patience = 10
    counter = 0
    
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for X_batch, y_batch in train_loader:
            X_batch = X_batch.to(device)
            y_prices = y_batch[:, :2].to(device)
            y_demand = y_batch[:, 2:].to(device)
            
            optimizer.zero_grad()
            prices_pred, demand_pred = model(X_batch)
            
            price_loss = criterion(prices_pred, y_prices)
            demand_loss = criterion(demand_pred, y_demand)
            loss = price_loss + demand_loss
            
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for X_val, y_val in val_loader:
                X_val = X_val.to(device)
                y_val_prices = y_val[:, :2].to(device)
                y_val_demand = y_val[:, 2:].to(device)
                
                val_prices_pred, val_demand_pred = model(X_val)
                val_price_loss = criterion(val_prices_pred, y_val_prices)
                val_demand_loss = criterion(val_demand_pred, y_val_demand)
                val_loss += val_price_loss + val_demand_loss
        
        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
            torch.save(model.state_dict(), 'best_model.pth')
        else:
            counter += 1
            if counter >= patience:
                print(f'Early stopping at epoch {epoch}')
                break
                
        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
    
    # Load best model
    model.load_state_dict(torch.load('best_model.pth'))
    return model

# Improved trading strategy using model predictions
def smart_trading_strategy(model, df_nn_enhanced, defer_demands):
    storage = [0]
    profit = 0
    profit_over_time = []
    actions = []
    
    # Track deferable demands
    pending_deferables = defer_demands.copy()
    satisfied_deferables = []
    
    # Get model predictions for the whole day
    model.eval()
    with torch.no_grad():
        X = torch.tensor(df_nn_enhanced.values, dtype=torch.float32)
        prices_pred, demand_pred = model(X)
        prices_pred = prices_pred.numpy()
        demand_pred = demand_pred.numpy()
    
    for tick in range(len(df_nn_enhanced)):
        current_storage = storage[-1]
        price_now = df_nn_enhanced['avg_buy_price'].iloc[tick]
        sell_price = df_nn_enhanced['avg_sell_price'].iloc[tick]
        
        # Get active deferables for this tick
        active_deferables = [d for d in pending_deferables 
                           if d['start'] <= tick <= d['end']]
        
        # Sort deferables by urgency (closest deadline first)
        active_deferables.sort(key=lambda x: x['end'])
        
        # Calculate predicted prices and demands
        pred_buy_price = prices_pred[tick][0]
        pred_sell_price = prices_pred[tick][1]
        pred_demand = demand_pred[tick][0]
        
        # Priority 1: Handle immediate deferable deadlines
        immediate_deferables = [d for d in active_deferables if d['end'] == tick]
        for defer in immediate_deferables:
            if current_storage >= defer['energy']:
                current_storage -= defer['energy']
            else:
                buy_amt = defer['energy'] - current_storage
                profit -= buy_amt * price_now
                current_storage = 0
            pending_deferables.remove(defer)
            satisfied_deferables.append(defer)
        
        # Priority 2: Charge from solar if available and storage permits
        sun = df_nn_enhanced['sunlight'].iloc[tick]
        if sun > 30 and current_storage < max_storage:
            charge_amt = min(max_storage - current_storage, sun/10)
            current_storage += charge_amt
            actions.append('charge_solar')
        
        # Priority 3: Buy energy if price is good and we have upcoming demands
        remaining_deferables = [d for d in pending_deferables if d['start'] <= tick]
        upcoming_demand = sum(d['energy'] for d in remaining_deferables)
        
        if price_now < pred_buy_price and current_storage < max_storage:
            target_storage = min(max_storage, current_storage + upcoming_demand)
            buy_amt = min(5, target_storage - current_storage)
            if buy_amt > 0:
                current_storage += buy_amt
                profit -= buy_amt * price_now
                actions.append('buy')
        
        # Priority 4: Satisfy non-urgent deferables if conditions are good
        if sun > 30 or price_now < pred_buy_price:
            for defer in active_deferables:
                if current_storage >= defer['energy']:
                    current_storage -= defer['energy']
                    pending_deferables.remove(defer)
                    satisfied_deferables.append(defer)
        
        # Priority 5: Sell excess if price is good
        if sell_price > pred_sell_price and current_storage > min_storage + upcoming_demand:
            sell_amt = current_storage - (min_storage + upcoming_demand)
            if sell_amt > 0:
                current_storage -= sell_amt
                profit += sell_amt * sell_price
                actions.append('sell')
        
        storage.append(current_storage)
        profit_over_time.append(profit)
    
    return storage[1:], profit_over_time, actions, satisfied_deferables, pending_deferables

# Main execution
if __name__ == "__main__":
    # Prepare data
    df_nn = load_historical_data(days=1000)
    df_nn_enhanced = prepare_training_data(df_nn)
    
    # Create and train model
    input_dim = df_nn_enhanced.shape[1]
    model = EnhancedDualPredictionNet(input_dim)
    
    # Create data loaders
    X = df_nn_enhanced.values
    y = np.column_stack([
        df_nn_enhanced['avg_buy_price'],
        df_nn_enhanced['avg_sell_price'],
        df_nn_enhanced['demand']
    ])
    
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
    
    train_dataset = torch.utils.data.TensorDataset(
        torch.tensor(X_train, dtype=torch.float32),
        torch.tensor(y_train, dtype=torch.float32)
    )
    val_dataset = torch.utils.data.TensorDataset(
        torch.tensor(X_val, dtype=torch.float32),
        torch.tensor(y_val, dtype=torch.float32)
    )
    
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32)
    
    # Train model
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    model = train_model(model, train_loader, val_loader, criterion, optimizer)
    
    # Get deferable demands
    defer_demands = get_deferable_demands()
    
    # Run trading strategy
    storage, profit_over_time, actions, satisfied, pending = smart_trading_strategy(
        model, df_nn_enhanced, defer_demands
    )
    
    # Plot results
    plot_results(storage, profit_over_time, actions, satisfied, pending)

Epoch 0, Train Loss: 5305.0897, Val Loss: 2180.8022
Epoch 10, Train Loss: 4143.7247, Val Loss: 2090.4385
Epoch 20, Train Loss: 4389.5886, Val Loss: 1924.8319
Epoch 30, Train Loss: 3668.9598, Val Loss: 1706.0028
Epoch 40, Train Loss: 3140.6182, Val Loss: 1405.8239
Epoch 50, Train Loss: 1966.9323, Val Loss: 825.4696
Epoch 60, Train Loss: 1270.9790, Val Loss: 519.9533
Epoch 70, Train Loss: 879.9182, Val Loss: 392.0804
Early stopping at epoch 77


NameError: name 'get_deferable_demands' is not defined

In [3]:
def plot_results(storage, profit_over_time, actions, satisfied, pending):
    """Plot the results of the trading strategy"""
    import matplotlib.pyplot as plt
    
    # Create figure with 3 subplots
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(15, 15))
    
    # Plot 1: Storage Level
    ax1.plot(storage, label='Storage Level', color='blue')
    ax1.axhline(y=25, color='r', linestyle='--', label='Min Storage')
    ax1.axhline(y=50, color='g', linestyle='--', label='Max Storage')
    ax1.set_title('Storage Level Over Time')
    ax1.set_xlabel('Tick')
    ax1.set_ylabel('Energy (J)')
    ax1.legend()
    ax1.grid(True)